In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# parameters
E = 10
r = 0.1
sigma = 0.4
T = 4/12
S_max = 100
N_plus = 50
N_minus = 0
N = 100
dt = T/N
dj = S_max/N_plus #2
omega = 1.2
tol = 0.00001

In [3]:
alpha = np.zeros(N_plus - N_minus + 1)
beta = np.zeros(N_plus - N_minus + 1)
gamma = np.zeros(N_plus - N_minus + 1)
for i in range(N_minus,N_plus+1): #0 to 51
    
    alpha[i] = 0.25*dt*((sigma*i)**2 + r*i)
    beta[i] = 0.5*dt*((sigma*i)**2 + r*i)
    gamma[i] = 0.25*dt*((sigma*i)**2 - r*i)

In [4]:
len(alpha)

51

In [5]:
def payoff_put(S,E):
    
    return np.maximum(E-S,0)

In [6]:
def payoff_call(S,E):
    return np.maximum(S-E,0)

In [7]:
def PSOR_Solver_American(american,g,Z_american,alpha,beta,gamma,N_minus,N_plus,omega,tol):
    
    american[N_minus] = g[N_minus]
    american[N_plus] = g[N_plus]
    
    error = 10000
    while error > tol:
        error = 0
        for i in range(N_minus+1,N_plus):
            y = (Z_american[i] + alpha[i]*american[i+1] + gamma[i]*american[i-1])/(1+beta[i])
            y = np.maximum(g[i],(american[i]+ omega*(y-american[i])))
            error = error + (american[i]- y)**2
            american[i] = y
    return american

In [8]:
def PSOR_Solver_European(european,g,Z_european,alpha,beta,gamma,N_minus,N_plus,omega,tol):
    
    european[N_minus] = g[N_minus]
    european[N_plus] = g[N_plus]
    
    error = 10000
    while error > tol:
        error = 0
        for i in range(N_minus+1,N_plus):
            y = (Z_european[i] + alpha[i]*european[i+1] + gamma[i]*european[i-1])/(1+beta[i])
            y = european[i]+ omega*(y-european[i])
            error = error + (european[i]- y)**2
            european[i] = y
    return european

In [9]:
def Put(alpha,beta,gamma,dj,E,N_plus,N_minus,tol,omega):
    
    size = N_plus - N_minus + 1 #51
    
    american = np.zeros((size))
    european = np.zeros((size))
    g = np.zeros((size))
    Z_american = np.zeros((size))
    Z_european = np.zeros((size))
    Sx = np.zeros((size))
    
    for i in range(N_minus,N_plus+1):
        Sx[i] = i*dj
        american[i] = payoff_put(Sx[i],E)
        european[i] = payoff_put(Sx[i],E)
        
    
    for n in range(N):
        
        for j in range(N_minus+1, N_plus):
            
            g[j] = payoff_put(Sx[j],E)
            
            Z_american[j] = alpha[j]*american[j+1] + (1-beta[j])*american[j] + gamma[j]*american[j-1]
            
            Z_european[j] = alpha[j]*european[j+1] + (1-beta[j])*european[j] + gamma[j]*european[j-1]
        
        g[N_minus] = payoff_put(Sx[N_minus],E)
        g[N_plus] = payoff_put(Sx[N_plus],E)
        
        american[N_minus] = g[N_minus]
        american[N_plus] = g[N_plus]
        
        european[N_minus] = g[N_minus]
        european[N_plus] = g[N_plus]
        
        american = PSOR_Solver_American(american,g,Z_american,alpha,beta,gamma,N_minus,N_plus,omega,tol)
        european = PSOR_Solver_European(american,g,Z_european,alpha,beta,gamma,N_minus,N_plus,omega,tol)
        
    return american,european

        

In [10]:
am_eu_put = Put(alpha,beta,gamma,dj,E,N_plus,N_minus,tol,omega)

In [11]:
am_eu_put

(array([1.00000000e+01, 7.66814318e+00, 5.48551579e+00, 3.46027817e+00,
        1.71275404e+00, 5.80296836e-01, 1.70814429e-01, 4.79421773e-02,
        1.33520678e-02, 3.75970250e-03, 1.08023678e-03, 3.18110435e-04,
        9.62017984e-05, 2.98948766e-05, 9.54444650e-06, 3.12890935e-06,
        1.05237611e-06, 3.62811854e-07, 1.28085515e-07, 4.62592815e-08,
        1.70748779e-08, 6.43530642e-09, 2.47425078e-09, 9.69637965e-10,
        3.87003272e-10, 1.57190243e-10, 6.49273700e-11, 2.72536367e-11,
        1.16181130e-11, 5.02686506e-12, 2.20627866e-12, 9.81725478e-13,
        4.42654113e-13, 2.02150182e-13, 9.34589399e-14, 4.37237299e-14,
        2.06912015e-14, 9.90052388e-15, 4.78823067e-15, 2.33983317e-15,
        1.15489781e-15, 5.75589947e-16, 2.89573706e-16, 1.47007309e-16,
        7.52768739e-17, 3.88450477e-17, 2.01467236e-17, 1.04067095e-17,
        5.17634245e-18, 2.13801852e-18, 0.00000000e+00]),
 array([1.00000000e+01, 7.66814318e+00, 5.48551579e+00, 3.46027817e+00,
      